In [10]:
# Import libraries
import pandas as pd
import statsmodels as sm
import numpy as np
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
import matplotlib.pyplot as plt
from scipy import stats

In [11]:
# Upload

datafile = "/Users/justinpassek/Documents/6469_Caesars_-_v5_v6_v7_20181203_170357_2026719673.csv"
df = pd.read_csv(datafile,skiprows=30)

print(df)

        Advertiser              Dynamic Profile  \
0          Caesars  CET_2018_Dynamic_V6_160x600   
1          Caesars  CET_2018_Dynamic_V6_160x600   
2          Caesars  CET_2018_Dynamic_V6_160x600   
3          Caesars  CET_2018_Dynamic_V6_160x600   
4          Caesars  CET_2018_Dynamic_V6_160x600   
5          Caesars  CET_2018_Dynamic_V6_160x600   
6          Caesars  CET_2018_Dynamic_V6_160x600   
7          Caesars  CET_2018_Dynamic_V6_160x600   
8          Caesars  CET_2018_Dynamic_V6_160x600   
9          Caesars  CET_2018_Dynamic_V6_160x600   
10         Caesars  CET_2018_Dynamic_V6_160x600   
11         Caesars  CET_2018_Dynamic_V6_160x600   
12         Caesars  CET_2018_Dynamic_V6_160x600   
13         Caesars  CET_2018_Dynamic_V6_160x600   
14         Caesars  CET_2018_Dynamic_V6_160x600   
15         Caesars  CET_2018_Dynamic_V6_160x600   
16         Caesars  CET_2018_Dynamic_V6_160x600   
17         Caesars  CET_2018_Dynamic_V6_160x600   
18         Caesars  CET_2018_Dy

In [12]:
# Mutate
# change column names
new_names = {'Mercury Res Module - Sales : RES Step 5 (Confirmation Page): Total Conversions':'Conversions',
            'Feed 1 - Reporting dimension 2 value':'CTA',
            'Feed 1 - Reporting dimension 1 value':'Layout',
            'Feed 1 - Reporting dimension 4 value':'Strategy',
            'Dynamic Profile':'Dynamic_Profile',
            'Feed 1 - Reporting dimension 5 value':'Carousel_Style',
            'Feed 1 - Reporting dimension 6 value':'Animation_Style'}

df.rename(columns=new_names,inplace=True)
# add conversion rate column
df['ConvRate']=df.Conversions / df.Impressions

In [13]:
## Clean
## 5000 impression minimum
#df = df[df["Impressions"]>5000]
## There must be more conversions than impressions
df = df[df["Impressions"]>df["Conversions"]]
## all capitals for dynamic profile and deliminate
df['Dynamic_Profile'] = df['Dynamic_Profile'].str.upper()
df[['CET','Year','Dynamic','Version','Size']] = df['Dynamic_Profile'].str.split('_',5,expand=True)
## rm unidentifiable values
df = df[df.Layout != '(not set)']
#df = df[df.CTA != '(not set)']
#df = df[df.Layout != 'Test Headline']
#df = df[df.CTA != 'Test Headline']
df = df[df.Layout != '---']
#df = df[df.CTA != '---']
## rm nas
df = df.dropna(subset=["Layout", "Version"])
## narrow to relevant columns
#imp_columns = ["Image","CTA","ConvRate","Impressions","Conversions"]
#df = df[imp_columns]
print(df)

     Advertiser              Dynamic_Profile  \
8       Caesars  CET_2018_DYNAMIC_V6_160X600   
10      Caesars  CET_2018_DYNAMIC_V6_160X600   
11      Caesars  CET_2018_DYNAMIC_V6_160X600   
12      Caesars  CET_2018_DYNAMIC_V6_160X600   
13      Caesars  CET_2018_DYNAMIC_V6_160X600   
14      Caesars  CET_2018_DYNAMIC_V6_160X600   
15      Caesars  CET_2018_DYNAMIC_V6_160X600   
16      Caesars  CET_2018_DYNAMIC_V6_160X600   
18      Caesars  CET_2018_DYNAMIC_V6_160X600   
19      Caesars  CET_2018_DYNAMIC_V6_160X600   
21      Caesars  CET_2018_DYNAMIC_V6_160X600   
22      Caesars  CET_2018_DYNAMIC_V6_160X600   
23      Caesars  CET_2018_DYNAMIC_V6_160X600   
24      Caesars  CET_2018_DYNAMIC_V6_160X600   
26      Caesars  CET_2018_DYNAMIC_V6_160X600   
27      Caesars  CET_2018_DYNAMIC_V6_160X600   
28      Caesars  CET_2018_DYNAMIC_V6_160X600   
30      Caesars  CET_2018_DYNAMIC_V6_160X600   
31      Caesars  CET_2018_DYNAMIC_V6_160X600   
32      Caesars  CET_2018_DYNAMIC_V6_160

In [14]:
# # # V6 OG vs Zoom Layout
## narrow to relevant columns
imp_columns = ["Layout","Version","ConvRate","Impressions","Conversions"]
df = df[imp_columns]
#df = df[df['Version'] == 'V6']
print(df)

     Layout Version  ConvRate  Impressions  Conversions
8        OG      V6  0.000613       328111        201.0
10       OG      V6  0.000214        23362          5.0
11       OG      V6  0.000000         1714          0.0
12       OG      V6  0.000000         1556          0.0
13       OG      V6  0.000000         1479          0.0
14       OG      V6  0.000000           14          0.0
15       OG      V6  0.000000          190          0.0
16     Zoom      V6  0.001118        85900         96.0
18     Zoom      V6  0.000917        85053         78.0
19     Zoom      V6  0.000000          984          0.0
21     Zoom      V6  0.002601         1538          4.0
22     Zoom      V6  0.001018          982          1.0
23     Zoom      V6  0.000450       137764         62.0
24     Zoom      V6  0.000461       101980         47.0
26     Zoom      V6  0.000000        10972          0.0
27     Zoom      V6  0.000000        11175          0.0
28     Zoom      V6  0.000112         8965      

In [74]:
## "pivot" (pv) tables to see Conversion Rate for each var/var combination
#pvdf_xy = df.groupby(['Image','CTA'],as_index=False)[('Impressions','Conversions')].sum()
#pvdf_xy['ConvRate'] = pvdf.Conversions/pvdf.Impressions
#pvdf_x = df.groupby(['Image'],as_index=False)[('Impressions','Conversions')].sum()
#pvdf_x['ConvRate'] = pvdf_v0.Conversions/pvdf_v0.Impressions
#pvdf_y = df.groupby(['CTA'],as_index=False)[('Impressions','Conversions')].sum()
#pvdf_y['ConvRate'] = pvdf_v1.Conversions/pvdf_v1.Impressions
#print(pvdf_xy)
#print(pvdf_x)
#print(pvdf_y)

   Image                     CTA  Impressions  Conversions  ConvRate
0    ATL  Book Early<br>And Save        48592          6.0  0.000123
1    ATL       FALL GETAWAY SALE      1846158        887.0  0.000464
2    ATL               HOT RATES            0         43.0  0.000177
3    ATL            LOCALS OFFER         3969         28.0  0.000276
4    BAC  Book Early<br>And Save        44050         10.0  0.000319
5    BAC       FALL GETAWAY SALE       775259        221.0  0.000289
6    BAC               HOT RATES            0         18.0  0.000249
7    BAC            LOCALS OFFER       110901         34.0       NaN
8    CAC  Book Early<br>And Save        38986         10.0       NaN
9    CAC       FALL GETAWAY SALE       819349        218.0       NaN
10   CAC               HOT RATES            0         24.0       NaN
11   CAC            LOCALS OFFER          215         26.0       NaN
12   CLV       Dine #LikeACaesar          758         61.0       NaN
13   FLV  Sneak away this summer  

In [15]:
def conv_rate_pivot_tbl_xy(x,y):
    pvdf_xy = df.groupby([x,y],as_index=False)[('Impressions','Conversions')].sum()
    pvdf_xy['ConvRate'] = pvdf_xy.Conversions/pvdf_xy.Impressions
    return pvdf_xy
    
def conv_rate_pivot_tbl_x(x):
    pvdf_x = df.groupby([x],as_index=False)[('Impressions','Conversions')].sum()
    pvdf_x['ConvRate'] = pvdf_x.Conversions/pvdf_x.Impressions
    return pvdf_x

# Input args (columns)
print(conv_rate_pivot_tbl_xy(df.Layout,df.Version))
print(conv_rate_pivot_tbl_x(df.Layout))
print(conv_rate_pivot_tbl_x(df.Version))

  Layout Version  Impressions  Conversions  ConvRate
0     OG      V6      7728265       3543.0  0.000458
1     OG      V7      6757058       1461.0  0.000216
2   Zoom      V6     15227485       8556.0  0.000562
3   Zoom      V7     13970990       2854.0  0.000204
  Layout  Impressions  Conversions  ConvRate
0     OG     14485323       5004.0  0.000345
1   Zoom     29198475      11410.0  0.000391
  Version  Impressions  Conversions  ConvRate
0      V6     22955750      12099.0  0.000527
1      V7     20728048       4315.0  0.000208


In [110]:
## A/B test for 1 Group, 2 Levels
#pvdf_x = conv_rate_pivot_tbl_x(df.Layout)
#OG = pvdf_x[pvdf_x['Layout']=='OG']
#Zoom = pvdf_x[pvdf_x['Layout']=='Zoom']

#stats.ttest_ind(OG['ConvRate'], Zoom('ConvRate'))


TypeError: 'DataFrame' object is not callable

In [16]:
# 1-way anova test
formula = 'ConvRate ~ Layout'
model = ols(formula,data=df).fit()
aov_table = anova_lm(model, typ=2)
print(aov_table)

            sum_sq      df         F    PR(>F)
Layout    0.000076     1.0  8.952031  0.002788
Residual  0.033902  4019.0       NaN       NaN


In [23]:
def one_way_anova_test(x):
    formula = 'ConvRate ~ x'
    model = ols(formula,data=df).fit()
    aov_table = anova_lm(model, typ=2)
    return aov_table

print(one_way_anova_test(df.Layout))

            sum_sq      df         F    PR(>F)
x         0.000076     1.0  8.952031  0.002788
Residual  0.033902  4019.0       NaN       NaN


In [17]:
# 2-way anova test
pvdf_xy = conv_rate_pivot_tbl_xy(df.Layout,df.Version)

ConvRate_Total = sum(df.Conversions)/sum(df.Impressions)
formula = 'ConvRate ~ Layout + Version + Layout:Version'
model = ols(formula,data=df).fit()
aov_table = anova_lm(model, typ=1)
print(aov_table)

                    df    sum_sq   mean_sq          F    PR(>F)
Layout             1.0  0.000076  0.000076   8.985547  0.002738
Version            1.0  0.000123  0.000123  14.639885  0.000132
Layout:Version     1.0  0.000020  0.000020   2.407161  0.120860
Residual        4017.0  0.033759  0.000008        NaN       NaN


In [18]:
def two_way_anova_test(x,y):
    ConvRate_Total = sum(df.Conversions)/sum(df.Impressions)
    formula = 'ConvRate ~ x + y + x:y'
    model = ols(formula,data=df).fit()
    aov_table = anova_lm(model, typ=1)
    return aov_table

print(two_way_anova_test(df.Layout,df.Version))

              df    sum_sq   mean_sq          F    PR(>F)
x            1.0  0.000076  0.000076   8.985547  0.002738
y            1.0  0.000123  0.000123  14.639885  0.000132
x:y          1.0  0.000020  0.000020   2.407161  0.120860
Residual  4017.0  0.033759  0.000008        NaN       NaN


In [19]:
# extraction from table and output
p_v0 = aov_table.iloc[0,4]
p_v1 = aov_table.iloc[1,4]

if p_v0 < .05:
    print(aov_table.index.values[0], 'Result IS significant')
else:
    print(aov_table.index.values[0], 'Result IS NOT significant')

if p_v1 < .05:
    print(aov_table.index.values[1], 'Result IS significant')
else:
    print(aov_table.index.values[1], 'Result IS NOT significant')

Layout Result IS significant
Version Result IS significant
